In [21]:
import cvxpy as cp 
import numpy as np

In [22]:
meta_parameter = {
    'number of agents': 3,
    'number of resources': 2,
    'number of S scenario': 3,
    'number of T scenario': 3,
}

import random
def generate_scenario(meta_parameter):
    #S scenarios in second stage, T in extended stage
    n = meta_parameter['number of agents']

    S = meta_parameter['number of S scenario']
    T = meta_parameter['number of T scenario']
    scenario = {}

    pT=np.random.rand(S,T)
    pT=pT/sum(sum(pT))
    pT=pT.tolist()
    
    for j in range(S):
        scenario[j] = {}
        xS = np.random.rand(n)*0.6
        scenario[j]['xi_1'] =  xS
        scenario[j]['prob'] = sum(pT[j])
        
        for k in range(T):
            xT= np.random.rand(n)
            scenario[j][k]={'xi_2':xT,'prob':pT[j][k],'cond_prob':pT[j][k]/sum(pT[j])}
    return scenario 


scenario = generate_scenario(meta_parameter)


In [38]:
import pickle
with open('data', 'rb') as handle:
    meta_parameter = pickle.load(handle)
    scenario = pickle.load(handle)

def conver_2_np(meta_parameter, scenario):
    n = meta_parameter['number of agents']
    K = meta_parameter['number of resources']

    S = meta_parameter['number of S scenario']
    T = meta_parameter['number of T scenario']
    #S scenarios in second stage, T in extended stage
    for j in range(S):
        scenario[j]['xi_1'] = np.asarray(scenario[j]['xi_1'])
        for k in range(T):
            scenario[j][k]['xi_2'] = np.asarray(scenario[j][k]['xi_2'])
    return scenario

scenario = conver_2_np(meta_parameter, scenario)

In [19]:
import cvxpy as cp
from cvxpy import sum as csum
alpha=10
z = cp.Variable((10, 10))
y = cp.Variable(10)
lb_z = [z>=0]
lb_y = [y>=0]
extended_obj = {}
for t in range(10):
    extended_obj[t] = csum([4*((0.1*y[i]/alpha) - \
                                (0.1*y[i]/alpha)**2)  for i in range(10)]) 

In [25]:
def evaluation(x, alpha):
    if (x<=alpha/2):
        return 4*((x/alpha) - (x/alpha)**2)

In [100]:
from cvxpy import sum as csum
def subproblem(x_s, meta_parameter, scenario_s, bm_s = None):
    n = meta_parameter['number of agents']
    K = meta_parameter['number of resources']

    S = meta_parameter['number of S scenario']
    T = meta_parameter['number of T scenario']
    
    alpha = 2*K
    reallo_cost = 0.01
    
    w_s = sum(evaluation(scenario_s['xi_1'][i]*x_s[i], alpha) for i in range(n))
    
    scenario_extended = {}

    for t in range(T):
        scenario_extended[t] = np.asarray([scenario_s[t]['xi_2'][i]*(1-scenario_s['xi_1'][i])\
             for i in range(n)])
    

    prob_extended_list = [ scenario_s[j]['cond_prob'] for j in range(T)]

    z = cp.Variable((n, n))
    y = cp.Variable(n)
    lb_z = z>=0
    lb_y = y>=0

    
    # conditional probability of extended stage, conditioning on S
    y_div_alpha = [ cp.multiply(scenario_extended[t], y)/alpha for t in range(T)] 
    
    extended_obj = [4*(csum(y_div_alpha[t]) - cp.sum_squares(y_div_alpha[t])) for t in range(T)]
    extended_obj_exp = csum([extended_obj[t]*prob_extended_list[t] for t in range(T)])

    obj = cp.Maximize(w_s + extended_obj_exp  - reallo_cost*csum(z))

    
    reallocate =   y == cp.multiply(1-scenario_s['xi_1'],x_s) +\
                                     csum(z,axis=0)-csum(z,axis=1)
    constraints = [lb_z] + [lb_y] + [reallocate]
    sp = cp.Problem(obj, constraints)
    

    max_it = 1e2
    it = 0

    if bm_s is None:
        sp.solve(solver=cp.OSQP)
        if sp.status not in ["infeasible", "unbounded"]:
            
            feas = 1
            
            lmbd = reallocate.dual_value
            
            cut_subdiff = np.asarray([lmbd[i]*(1-scenario_s['xi_1'][i]) + 4*(1-2*(scenario_s['xi_1'][i]/alpha)*x_s[i])\
                *(scenario_s['xi_1'][i]/alpha) for i in range(n)])
            
            obj_val = obj.value
            val_t = [extended_obj[t].value for t in range(T)]
            return cut_subdiff, feas, obj_val, val_t
        else:
            raise Exception('risk neutral subproblem is infeasible')
    # if given a benchmark, then add in event cut 
    else:

        value_list_b = [ bm_s['r_s'] + bm_s[t]['r_t'] for t in range(T) ]
        # short_fall_b = {}
        # for t in range(T):
        #     short_fall_b[t] =  sum(bm_s[tt]['cond_prob']*max(value_list_b[t] - value_list_b[tt],0) \
        #                                                 for tt in range(T))
        shtf = sp.addVars(T, T, vtype=GRB.CONTINUOUS, name="shtf", lb=list(np.zeros((T,T))))
        shtf_def = sp.addConstrs( (shtf[t, tt] >= value_list_b[t] - w_s +\
             reallo_cost*z.sum()- extended_obj[tt] for t in range(T)\
                 for tt in range(T)), name = 'shtf_def')
        
        #Write order constraint as T ineq constr: E[(eta-X)+]<=E[(eta-Y)+], for eta=Y(\omega_1), ...
        
        #expected shortfall of benchmark: E[(eta_t - Y)+]
        exp_shtf_bm = [sum(prob_extended_list[tt] * max(0,value_list_b[t]-value_list_b[tt]) for tt in range(T)) for t in range(T)]
        
        prob_extended_edit = dict( [((t,tt),prob_extended_list[tt]) for tt in range(T) for t in range(T)] )
        
        ord_constr = sp.addConstrs( (shtf.prod(prob_extended_edit, t, '*') <= exp_shtf_bm[t] for t in range(T)), name = 'ord_constr' )
        sp.optimize()
        if sp.status == GRB.OPTIMAL:
            short_fall = {}
            for t in range(T):
                short_fall[t] = sum(bm_s[tt]['cond_prob']*max(value_list_b[t]-\
                    w_s-extended_obj[tt].getValue() ,0) for tt in range(T))
            # print(sp.status)

            lmbd = [reallocate[i].Pi for i in range(n)]
            
            cut_subdiff = [lmbd[i]*(1-scenario_s['xi_1'][i]) + 4*(1-2*(scenario_s['xi_1'][i]/alpha)*x_s[i])\
                *(scenario_s['xi_1'][i]/alpha) for i in range(n)]
            
            obj_val = obj.getValue()
            val_t = [extended_obj[t].getValue() for t in range(T)]

            return cut_subdiff, 1, obj_val, val_t

                
        elif (sp.status == GRB.INFEASIBLE) or (sp.status == GRB.INF_OR_UNBD):
            
            sp.remove(reallocate)
            
            u = sp.addVars(n, vtype=GRB.CONTINUOUS, name="u", lb = -GRB.INFINITY)
            u_abs = sp.addVars(n, vtype=GRB.CONTINUOUS, name="u_abs")
            
            reallocate_again = sp.addConstrs( (u[i] + y[i] == (1-scenario_s['xi_1'][i])*x[i] + z.sum('*',i)-z.sum(i,'*') for i in range(n)),\
                                                name = 'reallocate_again')
            
            abs_of_u = sp.addConstrs( (u[i] <= u_abs[i] for i in range(n)) , name = 'abs_of_u')
            abs_of_u_ = sp.addConstrs( (-u[i] <= u_abs[i] for i in range(n)) , name = 'abs_of_u_')
            
            obj_feas = qsum(u_abs)
            sp.setObjective(obj_feas, GRB.MINIMIZE)
            
            sp.optimize()
            if sp.status != GRB.OPTIMAL:
                raise Exception(f'feasible subproblem status code {sp.status}')

            cut_val = obj_feas.getValue()
            
            lmbd = [reallocate_again[i].Pi for i in range(n)]
            
            
            cut_subdiff = [lmbd[i]*(1-scenario_s['xi_1'][i]) +\
                    4*(1-2*(scenario_s['xi_1'][i]/alpha)*x[i])*(scenario_s['xi_1'][i]/alpha) for i in range(n)]
            
            
            return cut_subdiff, 0, cut_val, [-float('inf') for i in range(T)]

In [86]:
x_start = np.ones(meta_parameter['number of agents'])
print(subproblem( x_start, meta_parameter, scenario[0]))

1.4873887113449042 [0.31517089955925326, 0.52132910694517, 0.46317252767662076] [0.27858419 0.28858419 0.27858419]
[[ 2.66681112e-32  2.87951812e-02 -1.38778895e-23]
 [ 6.93899069e-24  2.66681112e-32 -6.93889881e-24]
 [ 1.38778895e-23  6.67511042e-01  2.66681112e-32]]
[ 9.18569505e-01  1.18644096e+00 -5.23532414e-23]
([0.31517089955925326, 0.52132910694517, 0.46317252767662076], 1, 1.4873887113449042, [0.8125192715585889, 0.9761227425706014, 0.5894806798686503])


In [106]:
import gurobipy as gp
from gurobipy import GRB

def masterproblem_guroby(x_start, meta_parameter, scenario, bm = None, max_it = 1e2):
    n = meta_parameter['number of agents']
    K = meta_parameter['number of resources']

    S = meta_parameter['number of S scenario']
    T = meta_parameter['number of T scenario']
    
    # if bm is none we consider risk-neutral solution
    if bm is None:
        bm = [None for i in range(S)]
    
    # nvm
    # assert S == len(scenario)-1, f" S not equal:"
    # assert T == len(scenario[0][0]), f"T not equal"

    prob_s = [ scenario[s]['prob'] for s in range(S)]
    
    prob_s_dict = dict([(s, prob_s[s]) for s in range(S)])
    
    mp=gp.Model("masterproblem")
    
    x = mp.addVars(n, vtype=GRB.CONTINUOUS, name="x", lb=list(np.zeros(n)))
    v = mp.addVars(S, vtype=GRB.CONTINUOUS, name="v", lb=-GRB.INFINITY, ub=[100 for i in range(S)])
    
    v_exp = v.prod(prob_s_dict)
    
    mp.setObjective(v_exp, GRB.MAXIMIZE)
    
    mp.addConstr( x.sum() <= K )
    
    mp.setParam('OutputFlag', 0)
    
    it = 0 #iteration
    
    x_it = x_start #n*1 vector
    cuts = {}
    i_cut = 0
    while (it<max_it):
        # print('it', it)
        optm_sec_stage = [0 for i in range(S)]
        
        feas_sec_stage = True
        
        v_t = []
        #obj cut & feas cut
        for s in range(S):
            # print('S',s)
            subdiff, feas, val, val_t = subproblem(x_it, meta_parameter, scenario[s], bm_s=bm[s])

            if feas==1:
                subdiff_dict = dict([(i,subdiff[i]) for i in range(n)])
                cuts[i_cut] = mp.addConstr( v[s] <= val + x.prod(subdiff_dict) - sum(subdiff[i]*x_it[i]\
                     for i in range(n)) )
                optm_sec_stage[s] = val
                # print("\n add a objective cut for scenario", 'it:', it, 'scenario' ,s, "\n")
                
                i_cut += 1
            elif feas==0:
                subdiff_dict = dict([(i,subdiff[i]) for i in range(n)])
                mp.addConstr( val + x.prod(subdiff_dict) - sum(subdiff[i]*x_it[i] for i in range(n)) <= 0 )
                
                feas_sec_stage = False
                optm_sec_stage[s] = -float('inf')
                print("\n add a feasbility cut for scenario", 'it:', it, 'scenario' ,s, "\n")
            v_t.append(val_t)
        
        mp.optimize()
        # print(mp.display())
        if mp.status != GRB.OPTIMAL:
            raise Exception(f'master problem status code {mp.status}')
        x_it_ = x_it
        x_it = [x[i].x for i in range(n)]
        
        if feas_sec_stage == False:
            it = it + 1
            continue;
        if sum([abs(x_it_[i]-x_it[i]) for i in range(n)])<0.1:
            break
             
        
        stop_crit = sum(abs(optm_sec_stage[s]-v[s].X) for s in range(S))
        
        # print(optm_sec_stage)
        # print([v[s].x for s in range(S)])
        if ( stop_crit < 1e-5 ):
            break;    
            
        it = it + 1
       
    if it<max_it:
        print(it)
        return x_it, [v[s].x for s in range(S)], v_t
    else:
        raise Exception('pre-mature as max iteration number reached at master problem')

In [50]:
scenario[0]

{'xi_1': array([0.05263531, 0.50986527, 0.33248896]),
 'prob': 0.25599837531142505,
 0: {'xi_2': array([0.6604385 , 0.60395005, 0.38346318]),
  'prob': 0.09906173712642562,
  'cond_prob': 0.3869623664834429},
 1: {'xi_2': array([0.99805407, 0.55393336, 0.31084044]),
  'prob': 0.011185969396687397,
  'cond_prob': 0.04369547026647936},
 2: {'xi_2': array([0.16514007, 0.89081346, 0.12463044]),
  'prob': 0.14575066878831205,
  'cond_prob': 0.5693421632500778}}

In [98]:
def masterproblem(x_start, meta_parameter, scenario, bm = None, max_it = 1e2):
    n = meta_parameter['number of agents']
    K = meta_parameter['number of resources']

    S = meta_parameter['number of S scenario']
    T = meta_parameter['number of T scenario']
    
    # if bm is none we consider risk-neutral solution
    if bm is None:
        bm = [None for i in range(S)]

    prob_s = np.asarray( [ scenario[s]['prob'] for s in range(S)])
    
    prob_s_dict = dict([(s, prob_s[s]) for s in range(S)])
    
    x = cp.Variable(n)
    v = cp.Variable(S)
    lb_x = x>=0
    ub_v = v<=10
    
    
    v_exp = prob_s@v
    obj = cp.Maximize(v_exp)

    resource = csum(x)<=K

    constraints = [lb_x]+[ub_v]+[resource]
    
    it = 0 #iteration
    
    x_it = x_start #n*1 vector
    cuts = {}
    i_cut = 0

    while (it<max_it):
        # print('it', it)
        optm_sec_stage = np.zeros(S)
        
        feas_sec_stage = True
        
        v_t = []
        #obj cut & feas cut
        for s in range(S):
            # print('S',s)
            subdiff, feas, val, val_t = subproblem(x_it, meta_parameter, scenario[s], bm_s=bm[s])

            if feas==1:
                
                cuts[i_cut] = v[s] <= val + x@subdiff - subdiff@x_it 
                constraints.append(cuts[i_cut])
                optm_sec_stage[s] = val
                # print("\n add a objective cut for scenario", 'it:', it, 'scenario' ,s, "\n")
                
                i_cut += 1

            elif feas==0:
                cuts[i_cut] =  val + x@subdiff - subdiff@x_it  <= 0 
                constraints.append(cuts[i_cut])
                feas_sec_stage = False
                optm_sec_stage[s] = -float('inf')
                print("\n add a feasbility cut for scenario", 'it:', it, 'scenario' ,s, "\n")
            v_t.append(val_t)
        mp = cp.Problem(obj, constraints)
        mp.solve()
        # print(mp.display())
        if mp.status in ["infeasible", "unbounded"]:
            raise Exception(f'master problem status: {mp.status}')
        x_it_ = x_it
        x_it = x.value
        
        if feas_sec_stage == False:
            it = it + 1
            continue;
        if sum([abs(x_it_-x_it) for i in range(n)])<0.01:
            print('x stop moving')
            break
             
        stop_crit = sum(abs(optm_sec_stage-v.value) for s in range(S))
        
        # print(optm_sec_stage)
        # print([v[s].x for s in range(S)])
        if ( stop_crit < 1e-5 ):
            break;    
            
        it = it + 1
       
    if it<max_it:
        print(it)
        return x_it, v.value, v_t
    else:
        raise Exception('pre-mature as max iteration number reached at master problem')

In [107]:
x, val_s, val_t = masterproblem_guroby(x_start, meta_parameter, scenario)
print(x)
print(val_s)

Restricted license - for non-production use only - expires 2023-10-25
6
[0.0, 1.1371941964495207, 0.8628058035504792]
[1.1529206443777125, 1.3048727953642367, 1.1468178112287024]
